In [2]:
import pandas as pd
import numpy as np
import os
import csv
from datetime import datetime
import pytz
import json

In [3]:
from matplotlib import pyplot as plt
%matplotlib inline

source: https://www.kaggle.com/mkechinov/ecommerce-behavior-data-from-multi-category-store?select=2019-Nov.csv

In [3]:
df = pd.read_csv("../datasets/2019-Nov.csv")#download the file and save it in the datasets folder

In [4]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,NaN,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
3,2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
4,2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2


In [5]:
df["event_type"].unique()

array(['view', 'cart', 'purchase'], dtype=object)

#### Select only one category & remove duplicates

In [6]:
df_el = df[df["category_code"]=="electronics.smartphone"]
df_el = df_el.drop_duplicates(["event_type","product_id","user_id"])

df_el.to_csv("../datasets/2019-Nov-eletronics.csv") #save only eletronics items into datasets

In [5]:
print("Nr. of rows: {}".format(len(df_el)))
print("Nr. of distinct users: {}".format(df_el["user_id"].nunique()))
print("Nr. of distinct products: {}".format(df_el["product_id"].nunique()))
print("Median of user interaction: {}".format(np.median(df_el.groupby("user_id")["product_id"].count())))

Nr. of rows: 8075719
Nr. of distinct users: 1575532
Nr. of distinct products: 1341
Median of user interaction: 3.0


### Consider top 10000 users by number of purchases

In [9]:
n = 10000
purch_by_users = df_el[df_el["event_type"] 
                       =="purchase"].groupby("user_id")["product_id"].nunique().reset_index(name="nr_purch")
top_n = list(purch_by_users.sort_values("nr_purch", ascending=False).head(n)["user_id"])

### First recommender: consider only purchase events

In [10]:
#for the first recommender we only consider purchases as interaction
df_el_purch = df_el[(df_el["event_type"]=="purchase") & (df_el["user_id"].isin(top_n))]

In [11]:
print("Nr. users: {}".format(len(df_el_purch["user_id"].unique())))
print("Nr. products: {}".format(len(df_el_purch["product_id"].unique())))

Nr. users: 10000
Nr. products: 721


### Split train/test (last m purchases are in test)

In [17]:
#take all purchases except last one in the train set:
train_test = df_el_purch.copy()
train_test.loc[:,"event_time"]= pd.to_datetime(train_test.loc[:,"event_time"])
train_test = train_test.sort_values(["user_id","event_time"])
train_test.loc[:,"event_rank"] = train_test.groupby("user_id")["event_time"].cumcount()+1

In [18]:
train_test_max = train_test.groupby("user_id")["event_rank"].max().reset_index(name="max_rank")
train_test = train_test.merge(train_test_max, on="user_id", how="inner")

In [19]:
m = 1
train = train_test[train_test["event_rank"]<=(train_test["max_rank"]-m)]
test = train_test[train_test["event_rank"]>(train_test["max_rank"]-m)]

In [20]:
print("Nr. users in train: {}".format(len(train["user_id"].unique())))
print("Nr. products in train: {}".format(len(train["product_id"].unique())))

Nr. users in train: 10000
Nr. products in train: 685


### Save to csv

In [21]:
data_folder = "../data/input_data/"

In [24]:
cols = ["event_time","event_type","product_id","category_id","category_code","brand",
       "price","user_id"]
train[cols].to_csv(os.path.join(data_folder, "2019-Nov-sample-train-eletronics-purch-10kusers.csv"), header=False)
test[cols].to_csv(os.path.join(data_folder, "2019-Nov-sample-test-eletronics-purch-10kusers.csv"), header=False)
test.reset_index()["user_id"].to_csv(os.path.join(data_folder, "2019-Nov-sample-test-eletronics-userid-10kusers.csv"), 
                       header=False)

### For second recommender (purchase, view): consider the top n users above and their purchases,views

In [25]:
df_el_p_v = df_el[((df_el["event_type"]=="purchase") | (df_el["event_type"]=="view"))  & (df_el["user_id"].isin(top_n))]

In [26]:
print("Nr. users: {}".format(len(df_el_p_v["user_id"].unique())))
print("Nr. products: {}".format(len(df_el_p_v["product_id"].unique())))

Nr. users: 10000
Nr. products: 1039


### Split train/test: use test set from previous recommender use all other purchase and views for train 

In [27]:
#test = pd.read_csv(os.path.join(data_folder, "2019-Nov-sample-test-eletronics-purch-10kusers.csv"),
#                   names=df_el_p_v.columns)

In [28]:
merge_on = ["product_id","user_id"]
train2 = df_el_p_v.merge(test[merge_on], on=merge_on, how="left", indicator=True)
# exclude user-product that are already in the test set (indipendent from the event type)
train2 = train2[train2["_merge"]=="left_only"].drop(columns="_merge")

In [29]:
print("Nr of rows in train: {}".format(len(train2)))
print("Nr. users in train: {}".format(len(train2["user_id"].unique())))
print("Nr. products in train: {}".format(len(train2["product_id"].unique())))

Nr of rows in train: 178360
Nr. users in train: 10000
Nr. products in train: 1038


### Save to csv

In [30]:
cols = ["event_time","event_type","product_id","category_id","category_code","brand",
       "price","user_id"]
train2.loc[:,"event_time"]= pd.to_datetime(train2.loc[:,"event_time"])
train2[cols].to_csv(os.path.join(data_folder, "2019-Nov-sample-train-eletronics-purch-view-10kusers.csv"), header=False)


### For third recommender (purchase, view, cart): consider the top n users above and their purchases,views, carts

In [31]:
df_el_p_v_c = df_el[(df_el["user_id"].isin(top_n))]

In [32]:
print("Nr. users: {}".format(len(df_el_p_v_c["user_id"].unique())))
print("Nr. products: {}".format(len(df_el_p_v_c["product_id"].unique())))

Nr. users: 10000
Nr. products: 1039


### Split train/test: use test set from previous recommender use all other purchase and views for train 

In [33]:
test = pd.read_csv(os.path.join(data_folder, "2019-Nov-sample-test-eletronics-purch-10kusers.csv"),
                   names=df_el_p_v.columns)

In [34]:
merge_on = ["product_id","user_id"]
train3 = df_el_p_v_c.merge(test[merge_on], on=merge_on, how="left", indicator=True)
# exclude user-product that are already in the test set (indipendent from the event type)
train3 = train3[train3["_merge"]=="left_only"].drop(columns="_merge")

In [35]:
print("Nr of rows in train: {}".format(len(train3)))
print("Nr. users in train: {}".format(len(train3["user_id"].unique())))
print("Nr. products in train: {}".format(len(train3["product_id"].unique())))

Nr of rows in train: 227076
Nr. users in train: 10000
Nr. products in train: 1038


### Save to csv

In [36]:
cols = ["event_time","event_type","product_id","category_id","category_code","brand",
       "price","user_id"]
train3.loc[:,"event_time"]= pd.to_datetime(train3.loc[:,"event_time"])
train3[cols].to_csv(os.path.join(data_folder, "2019-Nov-sample-train-eletronics-purch-view-cart-10kusers.csv"), header=False)
